# Scrape data from Fortune500.com

This program reads in html files downloaded from Fortune500.com and creates a Pandas DataFrame with information on assets, revenues, and profits for the top 500 companies (ranked by total revenues for their respective fiscal years).

## Imports

In [1]:
import pandas as pd
import bs4 # BeautifulSoup 4
import re
import platform

## Write out ENVIRONMENT.md file

Before moving on, generate an ENVIRONMENT.md file with details about computing environment and versions of imported dependencies.

This is a shortcut (and likely very inferior) to actually using something like a [Docker](https://www.docker.com/) container, or using  [virtualenv](https://pypi.org/project/virtualenv/) and [virtualenvwrapper](https://virtualenvwrapper.readthedocs.io/en/latest/).

But 

In [2]:
import sys
sys_info = sys.version
os = platform.platform()
python = platform.python_version()


with open('ENVIRONMENT.md', "w", encoding='utf8') as outfile:
    print("# Computation Environment and Dependencies", file=outfile)
    print("To help with replicability, below are the details about the Operating System, \
Python Environment, and Imported Libraries used in this analysis.", file=outfile)
    print("", file=outfile)
    print("## Operating System", file=outfile)
    print("{}".format(os), file=outfile)
    print("", file=outfile)
    print("## Python Details", file=outfile)
    print("", file=outfile)
    print("Python version == {}".format(sys_info), file=outfile)
    print("", file=outfile)    
    print("## Imported Libraries", file=outfile)
    print("", file=outfile)
    print("pandas version == {}".format(pd.__version__), file=outfile)
    print("bs4 version == {} (Beautiful Soup)".format(bs4.__version__), file=outfile)
    print("re version == {} (regular expression engine)".format(re.__version__), file=outfile)
    print("platform version == {}".format(platform.__version__), file=outfile)


### A Failed Attempt

This cell would be used if I could figure out how to get the full html file to download with the requests library. As is, the Fortune.com site only partially loads the data in the html file, and then fills it further when the page has been scrolled to the bottom.

In [3]:
#import requests
#fortune_url = "http://fortune.com/fortune500/"
#year = 2015
#fortune_suffix = "/list/filtered?sortBy=profits&first500"
#page = requests.get(fortune_url + str(year) + fortune_suffix)

### Some Success...
Because of the partal loading of the data, I had scroll to the bottom of each html file and then  manually download the html files.

I downloaded 12 separate html files, by year (2015-2018) and variable of interest.

These files are saved in the input\ directory

In [4]:
start_year = 2015
end_year = 2018

vars = ['revenues', 'profits', 'assets']

In [5]:
def scrape_Fortune500_data(var, year):
    '''
    Purpose: Get data from a Fortune500 html file.
    Parameters: 'var' -- can take 'revenues', 'profits', or 'assets'
                'year' -- current years available are 2015 - 2018
    Returns: A list with data for 'var'
    '''
           
    # class_dict: Dictionary with variables as keys and html span classes as the values
    class_dict = {'company' : 'column small-5 company-title',
                  'rank' : 'column small-2 company-rank ',    # Trailing space is necessary
                  'revenues' : 'column small-5 company-revenue',  # Same class for each of the vars
                  'profits' : 'column small-5 company-revenue',
                  'assets' : 'column small-5 company-revenue'}

    if var in class_dict:
        class_value = class_dict[str(var)]
    else:
        print ("KeyError: Accepted keys: ['company', 'rank', 'revenues', 'profits', 'assets']")        
    
    # ResultSets are BeautifulSoup objects, containing Tags (another BeautifulSoup object)
    result_set = html_file.findAll('span', {'class': class_value}) 
    # print (result_set[0:10])

    var_list = []
    for tag in result_set:
        str_tag = str(tag) # Convert BeautifulSoup Tag object to a string
        
        # Find item of interest
        item = str_tag[str_tag.index('>')+1 : str_tag.index('</span>')]
       
        # Clean up company strings with &s in their names 
        if (var == 'company'):
             item = re.sub('amp;', '', item) # '&' (AT&T, Procter & Gamble)
        # Clean up numeric data
        else:
            item = re.sub('(,)|(\$)|(\.0)','', item)
        
        # Add each (cleaned up) item to the list
        var_list.append(item)
    
    # Check that all 500 items were captured
    assert (len(var_list) == 500), "There were only {} records read in from {}.".format(len(var_list), filename) #[filename, len(var_list)]
    
    return var_list

In [6]:
# Define empty dictionaries
year_dict = {}
var_dict = {}

for year in range(start_year, end_year + 1):
    
    for var in vars:
        
        filename = "input\\Fortune500_" + var + "_" + str(year) + ".html"
        
        with open(filename, encoding='utf8') as infile:
            html_file = bs4.BeautifulSoup(infile, 'html.parser')
        
        # Scrape the data with function, defined above
        # Get rank, company name, and variable of interest from each file
        rank_list = scrape_Fortune500_data('rank', year)
        company_list = scrape_Fortune500_data('company', year)
        var_list = scrape_Fortune500_data(var, year)

        # Put results in variable dictionary
        var_dict[var] = pd.DataFrame({'year':year,
                                      'rank':rank_list, 
                                      'company':company_list,
                                       var:var_list})
 
    # Merge results into a single dataframe and put it into a year dictionary
    year_dict[year] = pd.merge(pd.merge(var_dict['revenues'],var_dict['profits'], on=['year', 'rank', 'company']),
                               var_dict['assets'], on=['year', 'rank', 'company'])

# Concatinate all the years into a single file
final_df = pd.concat(year_dict, ignore_index=True)

In [7]:
final_df.head()

,company,rank,revenues,year,profits,assets
0,Walmart,1,485651,2015,16363,203706
1,Exxon Mobil,2,382597,2015,32520,349493
2,Chevron,3,203784,2015,19241,266026
3,Berkshire Hathaway,4,194673,2015,19872,526186
4,Apple,5,182795,2015,39510,231839


In [8]:
final_df.tail()

,company,rank,revenues,year,profits,assets
1995,Polaris Industries,496,5504.8,2018,172.5,3089.6
1996,Pioneer Natural Resources,497,5455,2018,833,17003
1997,ABM Industries,498,5453.6,2018,3.8,3812.6
1998,Vistra Energy,499,5430,2018,-254,14600
1999,Cintas,500,5428.9,2018,480.7,6844.1


In [9]:
# Write out final df
final_df.to_csv("output\\Fortune500_" + str(start_year) + "_" + str(end_year) + ".csv", 
                index=False,
                columns=['year', 'company', 'rank'] + vars)